In [322]:
import math
import numpy as np
np.float = float
np.int   = int
body_imu = [1.72, -15.86, -0.08]
crotch_imu = [1.86, -14.88, 0.57]
# body_imu = [-1.10, -1.36, 0.90]
# crotch_imu = [-1.01, -0.62, -14.12]
body_imu_rad    = np.deg2rad(body_imu)
crotch_imu_rad  = np.deg2rad(crotch_imu)
# --- helper 함수들 ---
#회전 행렬 만들기
#x축으로 roll, y축으로 pitch, z축으로 yaw
def rpy_to_mat(roll, pitch, yaw): 
    
    Rx = np.array([[1, 0, 0],
                   [0, math.cos(roll), -math.sin(roll)],
                   [0, math.sin(roll),  math.cos(roll)]])
    Ry = np.array([[ math.cos(pitch), 0, math.sin(pitch)],
                   [0, 1, 0],
                   [-math.sin(pitch), 0, math.cos(pitch)]])
    Rz = np.array([[math.cos(yaw), -math.sin(yaw), 0],
                   [math.sin(yaw),  math.cos(yaw), 0],
                   [0, 0, 1]])
    return Rz @ Ry @ Rx


def axis_angle_to_mat(axis, angle):
    axis = np.array(axis) / np.linalg.norm(axis) #단위 벡터로 만들기
    x,y,z = axis #축을 x,y,z로 나누기
    c, s = math.cos(angle), math.sin(angle) #삼각함수 각도 구하기
    C = 1-c #1-cos 저장
    # Rodrigues 회전 공식 적용
    return np.array([
      [x*x*C + c,   x*y*C - z*s, x*z*C + y*s],
      [y*x*C + z*s, y*y*C + c,   y*z*C - x*s],
      [z*x*C - y*s, z*y*C + x*s, z*z*C + c  ]
    ])

In [323]:
import json
from urdfpy import URDF

ROOT_PATH = '/home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001'
# 1. URDF와 data.json 파일 경로
URDF_PATH   = '/home/scilab/Documents/teleoperation/avp_teleoperate/assets/g1_inspire_hand_description/g1_29dof_rev_1_0_with_inspire_hand_FTP.urdf'    # URDF가 담긴 .txt 파일
DATA_PATH   = '/home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001/data.json'   # data.json 파일


finger_map = {'thumb':5, 'thumb2':4, 'index':3, 'middle':2, 'ring':1, 'pinky':0}

LEFTRIGHT = 'left'
WHICH_FINGER = 'thumb'
FRAME = 942

names = [LEFTRIGHT, 'shoulder', 'elbow', 'wrist', 'pitch', 'roll', 'yaw']   #얘는 list로 비교하면서 바꿨고 (아래 for i, joint in enumerate(joints) 부분에서)
handmap = {
                f'k{LEFTRIGHT.capitalize()}HandThumbRotation' : f'{LEFTRIGHT}_thumb_1_joint',
                f'k{LEFTRIGHT.capitalize()}HandThumbBend' : f'{LEFTRIGHT}_thumb_3_joint',
                f'k{LEFTRIGHT.capitalize()}HandIndex' : f'{LEFTRIGHT}_index_1_joint',
                f'k{LEFTRIGHT.capitalize()}HandMiddle' : f'{LEFTRIGHT}_middle_1_joint',
                f'k{LEFTRIGHT.capitalize()}HandRing' : f'{LEFTRIGHT}_ring_1_joint',
                f'k{LEFTRIGHT.capitalize()}HandPinky' : f'{LEFTRIGHT}_little_1_joint',
            }

# 2. URDF 로드
robot = URDF.load(URDF_PATH)

# 3. data.json 로드
with open(DATA_PATH, 'r') as f:
    data = json.load(f)


joints  = data['info']['joint_names'][f'{LEFTRIGHT}_arm']  # 7개 joint names list       여기부터

for i, joint in enumerate(joints):
    tmp_name = ""

    joint = joint[1:]
    for name in names:
        find_res = joint.lower().find(name)
        if find_res == -1:
            continue
        else:
            tmp_name = tmp_name + joint[:find_res+len(name)] + '_'
            joint = joint[find_res+len(name):]
    
    joints[i] = (tmp_name[:-1] + '_joint').lower()

# 4. idx=0의 left_arm 관절각(qpos)과 joint 이름 목록 가져오기
qpos    = data['data'][FRAME]['states'][f'{LEFTRIGHT}_arm']['qpos']                         #여기까지는 arm을 뽑는부분

hand_qpos  = data['data'][FRAME]['states'][f'{LEFTRIGHT}_hand']['qpos']                 #여기부터
hand_joints = data['info']['joint_names'][f'{LEFTRIGHT}_hand']
hand_joints = [handmap[x] for x in hand_joints]                                       #여기까지는 hand joints 정리해두는부분 이중에 한개만 뽑아서 넣을거임

# 5. joint name → angle 매핑
q_dict = {name: angle for name, angle in zip(joints, qpos)}

q_dict[hand_joints[finger_map[WHICH_FINGER]]] = hand_qpos[finger_map[WHICH_FINGER]]       #hand q_dict에 추가

T_world_base = np.eye(4)
T_world_base[:3,:3] = rpy_to_mat(*body_imu_rad)
T_world_base[:3,3] = [0,0,crotch_imu_rad[2]]
# 6. forward kinematics 계산  
# ee_link = f'{LEFTRIGHT}_wrist_yaw_link'     # wrist뽑을때 uncomment, hand 뽑을때 comment
ee_link = hand_joints[finger_map[WHICH_FINGER]][:-6]                    # hand 뽑을때 uncomment, wrist 뽑을때 comment
print(ee_link)
# link_fk()는 각 링크별로 월드 기준 4×4 변환행렬을 반환
fk = robot.link_fk(q_dict)
# for link_obj, T in fk.items():
#     print(link_obj.name)
# T_ee_world = fk[ee_link]   # (4,4) 행렬
ee_link_obj  = robot.link_map[ee_link]    # 이름→Link 객체 얻기
T_ee_world   = fk[ee_link_obj]                 # 올바르게 인덱싱!

# 7. 위치(translation) 추출
pos_ee_world = T_world_base@T_ee_world 
print('Left arm end-effector position in world frame:', pos_ee_world)


left_thumb_1
Left arm end-effector position in world frame: [[-0.73731188  0.36801076  0.56651502  0.21837372]
 [-0.54228004 -0.82251683 -0.17145966  0.18312187]
 [ 0.40286914 -0.43362904  0.80601633  0.22932673]
 [ 0.          0.          0.          1.        ]]


In [324]:
# cam = robot['d435I_joint']
T_world_cam = np.eye(4)
T_world_cam[:3,:3] = rpy_to_mat(0, 0.8307767239493009, 0)
T_world_cam[:3, 3] = [0.0576235, 0.01753, 0.42987]   # :contentReference[oaicite:0]{index=0}


# 5) camera→left_wrist 변환
T_cam_ee = np.linalg.inv(T_world_cam) @ pos_ee_world

# 6) translation 성분이 바로 “카메라 기준” 손끝 좌표
pos_cam_ee = T_cam_ee[:3, 3]
print("Left wrist position in D435 camera frame:", pos_cam_ee)

Left wrist position in D435 camera frame: [ 0.2564865   0.16559187 -0.01651995]


In [325]:
import cv2
import numpy as np

# 1) Intrinsics
width, height = 848, 480
fx = 426.7954
fy = 426.7954
# cx = width  / 2  # 424
# cy = height / 2  # 320
# fx=605.9278 
# fy=606.1917 
cx = width  / 2  
cy = height / 2  
# 2) pos_cam_ee: (X, Y, Z)  
#    X: 오른쪽(+), Y: 카메라 앞쪽(+), Z: 위쪽(+)
# print(pos_cam_ee)
# Xc =  pos_cam_ee[0]      
# Yc =  pos_cam_ee[1]      
# Zc =  pos_cam_ee[2]     
# Xc, Yc, Zc = T_cam_ee[:3,3]
X_urdf, Y_urdf, Z_urdf = pos_cam_ee

# 1) 축 재배열
Xc =  X_urdf
Yc = -Y_urdf
Zc = -Z_urdf
# 2) 투영
u = fx * (Yc / Xc) + cx
v = fy * (Zc / Xc) + cy

# 소수점→정수 픽셀 좌표
print(u,v)
pt = (int(round(u)), int(round(v)))
print(pt)
# 4) 이미지 읽고 표시  
img = cv2.imread("/home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001/colors/000942_color_0.jpg")
print(FRAME)
if img is None:
    raise RuntimeError("Unable to load image.")
cv2.circle(img, pt, 8, (255,0,0), thickness=-1)  # 빨간 점
cv2.putText(img, f"{pt}", (pt[0]+10, pt[1]),
            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,255), 2)

# 5) 결과 저장 (Jupyter에서 imshow 대신)
cv2.imwrite(f"projected_pointindex2{FRAME}.png", img)
print("Saved to projected_point.png")


148.45393388110017 267.4893201712274
(148, 267)
942
Saved to projected_point.png


In [326]:
import math
import json
import cv2
import numpy as np
from urdfpy import URDF

# ------------------- 설정 영역 -------------------
ROOT_PATH = '/home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001'
# 1. URDF와 data.json 파일 경로
URDF_PATH   = '/home/scilab/Documents/teleoperation/avp_teleoperate/assets/g1_inspire_hand_description/g1_29dof_rev_1_0_with_inspire_hand_FTP.urdf'    # URDF가 담긴 .txt 파일
DATA_PATH   = '/home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001/data.json'   # data.json 파일




FRAME        = 942
LEFTRIGHT    = 'left'       # 'left' 또는 'right'
WHICH_FINGER = 'index'      # 'thumb','index','middle','ring','pinky'

body_imu_deg   = [1.72, -15.86, -0.08]
crotch_imu_deg = [1.86, -14.88, 0.57]

WIDTH, HEIGHT = 848, 480
# FX, FY        = 426.7954, 426.7954
FX=300.9278 
FY=300.1917 
CX, CY        = WIDTH/2, HEIGHT/2

# JSON에서 뽑힌 손가락 순서 → URDF joint 이름 매핑
finger_map = {'thumb':5, 'index':3, 'middle':2, 'ring':1, 'pinky':0}
handmap = {
    'kLeftHandThumbRotation': 'left_thumb_1_joint',
    'kLeftHandThumbBend':     'left_thumb_3_joint',
    'kLeftHandIndex':         'left_index_1_joint',
    'kLeftHandMiddle':        'left_middle_1_joint',
    'kLeftHandRing':          'left_ring_1_joint',
    'kLeftHandPinky':         'left_little_1_joint',
    'kRightHandThumbRotation':'right_thumb_1_joint',
    'kRightHandThumbBend':    'right_thumb_3_joint',
    'kRightHandIndex':        'right_index_1_joint',
    'kRightHandMiddle':       'right_middle_1_joint',
    'kRightHandRing':         'right_ring_1_joint',
    'kRightHandPinky':        'right_little_1_joint',
}

def rpy_to_mat(roll, pitch, yaw):
    Rx = np.array([[1,0,0],
                   [0,math.cos(roll),-math.sin(roll)],
                   [0,math.sin(roll), math.cos(roll)]])
    Ry = np.array([[ math.cos(pitch),0, math.sin(pitch)],
                   [0,1,0],
                   [-math.sin(pitch),0, math.cos(pitch)]])
    Rz = np.array([[math.cos(yaw),-math.sin(yaw),0],
                   [math.sin(yaw), math.cos(yaw),0],
                   [0,0,1]])
    return Rz @ Ry @ Rx

import re
def camel_to_urdf_joint(name):
    # kLeftShoulderPitch → left_shoulder_pitch_joint
    base = name[1:] if name[0].lower()=='k' else name
    snake = re.sub(r'(?<!^)(?=[A-Z])','_',base).lower()
    snake = snake.replace('wristyaw','wrist_yaw')
    return snake + '_joint'

# 1) URDF, JSON 로드
robot = URDF.load(URDF_PATH)
with open(DATA_PATH,'r') as f:
    data = json.load(f)

# 2) JSON에서 arm / hand joint 뽑기
arm_names  = data['info']['joint_names'][f'{LEFTRIGHT}_arm']
arm_qpos   = data['data'][FRAME]['states'][f'{LEFTRIGHT}_arm']['qpos']
hand_names = data['info']['joint_names'][f'{LEFTRIGHT}_hand']
hand_qpos  = data['data'][FRAME]['states'][f'{LEFTRIGHT}_hand']['qpos']

# 3) arm → URDF joint mapping
q_dict = {}
for name, q in zip(arm_names, arm_qpos):
    q_dict[camel_to_urdf_joint(name)] = q

# 4) 손가락 하나만 handmap으로 추가
idx = finger_map[WHICH_FINGER]
json_hand_name = hand_names[idx]
urdf_hand_j    = handmap[json_hand_name]
q_dict[urdf_hand_j] = hand_qpos[idx]

# 5) Base(IMU) → World
body_imu   = np.deg2rad(body_imu_deg)
crotch_imu = np.deg2rad(crotch_imu_deg)
T_world_base = np.eye(4)
T_world_base[:3,:3] = rpy_to_mat(*body_imu)

# 6) Forward Kinematics
fk = robot.link_fk(q_dict)
ee_link = f'{LEFTRIGHT}_wrist_yaw_link'
T_ee_world = fk[robot.link_map[ee_link]]

# 7) World → Camera (d435_link)
T_world_cam = fk[robot.link_map['d435_link']]
T_cam_ee    = np.linalg.inv(T_world_cam) @ (T_ee_world @ T_world_base)
Xc, Yc, Zc  = T_cam_ee[:3,3]

# 8) 픽셀 투영
u = FX * (-Yc/Xc) + CX
v = FY * (-Zc/Xc) + CY
pt = (int(round(u)), int(round(v)))
print("투영된 픽셀 좌표:", pt)

# 9) 이미지에 그리기 및 저장
img_rel = data['data'][FRAME]['colors']['color_0']
img     = cv2.imread(f"{ROOT_PATH}/{img_rel}")
cv2.circle(img, pt, 8, (0,0,255), -1)
cv2.putText(img, str(pt), (pt[0]+10,pt[1]),
            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0),2)
out = f"projected_{LEFTRIGHT}_{WHICH_FINGER}_{FRAME}.png"
cv2.imwrite(out, img)
print("이미지 저장:", out)


투영된 픽셀 좌표: (274, 364)
이미지 저장: projected_left_index_942.png
